# Markr.AI Data Generation

`@ Developer : DopeorNope (Seungyoo Lee), Kyujinpy (Kyunjin Han)`

`@ Updata Date: 2024.03.22`

`@ Description :` 

- AI Hub 데이터와, Public Data를 통해서 Fine-tuning용 데이터를 생성하는 코드입니다.
    
    - 데이터를 생성하기 위해서는 기본 경로에,  '도서자료 요약', '문서요약 텍스트', '018.논문자료 요약 데이터', '048.일반상식 문장 생성 데이터', 'KorQuAD_v1.0_train.json' 데이터를 넣어주시면 됩니다.

    - Chat GPT가 아닌, License에 제약이 없는 방법론으로 기존의 데이터들을 생성해내는 것을 학습 세트로 하여 `Custom LLM을 개발` 할 수 있는 것을 목표로 하여 Code를 공유하기로 하였습니다.

    - 어떤 말뭉치 데이터(corpus)로도 코드르 조금만 수정하면 원하시는 도메인에 맞는 데이터셋을 생성 할 수 있습니다.     

# 일반 상식 문장생성데이터 Corpus로 데이터 생성하기

- `KoCommercial-Dataset.ipynb`에서 생성한 논문자료요약 Corpus를 통해서 fine-tuning 데이터셋을 생성하는 코드입니다.


- 위의 ipynb파일의 파이프라인을 실행 후 사용하시길 권장합니다.

In [ ]:
import pandas as pd
from datasets import dataset_dict, load_dataset,Dataset,DatasetDict
from tqdm import tqdm
import random
import string
import re

In [2]:
new_df=pd.read_parquet('./13/일반상식문장생성데이터.parquet')

In [3]:
filtered_df = new_df

In [4]:
for i in range(len(filtered_df)):
    
    filtered_df['text'][i] = filtered_df['text'][i].strip()
    
    try:
        if filtered_df['text'][i].endswith('. '):
            filtered_df['text'][i]=filtered_df['text'][i][:-1]

        elif filtered_df['text'][i].endswith('.'):
            pass

        elif filtered_df['text'][i].endswith(' .'):
            print(i, '번째 문장')
            print(filtered_df['text'][i])
            filtered_df['text'][i]=filtered_df['text'][i][:-2]+'.'

            print(filtered_df['text'][i])
            

        else:
            print(i, '번째 문장')
            print(filtered_df['text'][i])


            if filtered_df['text'][i].endswith(' '):
                filtered_df['text'][i]=filtered_df['text'][i][:-1]+'.'
                

            else:
                filtered_df['text'][i]=filtered_df['text'][i]+'.'
                
            
            print(filtered_df['text'][i])


    except Exception as e:
        print(e)
        print(i)
        print(filtered_df['text'][i])

        print('######################################################')

In [5]:
remove_index = []
for i in range(len(filtered_df)):
    text = filtered_df['text'][i]
    
    if '�' in text:
        remove_index.append(i)
    elif '삭제.' in text[-5:]:
        remove_index.append(i)

In [6]:
filtered_df.drop(remove_index, inplace=True)
filtered_df.reset_index(drop=True, inplace=True)

In [7]:
filtered_df

,text,doc_id,domain
0,아버지께 들은 이야기로 인해서 소는 나에게 경계 대상이다.,AIHUB_일반상식문장생성데이터_0,None
1,마을은 축제를 구경하기 위해 찾아온 사람들로 붐빈다.,AIHUB_일반상식문장생성데이터_1,None
2,그들은 우리의 시각을 자극하는 요소들을 정비했다.,AIHUB_일반상식문장생성데이터_2,None
3,나는 옛날 대학 시절부터 오늘날까지 이 책을 읽어 왔다.,AIHUB_일반상식문장생성데이터_3,None
4,농부는 농장에서 키우는 말에 고삐를 달아 끌고 갔다.,AIHUB_일반상식문장생성데이터_4,None
...,...,...,...
406491,총을 든 그들은 그에게 총격을 가해 왔다.,AIHUB_일반상식문장생성데이터_406491,None
406492,그가 직장을 그만두고 나서 힘들게 생활하고 있다.,AIHUB_일반상식문장생성데이터_406492,None
406493,그는 그 일을 아무 생각 없이 그대로 따라했다.,AIHUB_일반상식문장생성데이터_406493,None
406494,이번 추석에도 멀리 해외에 계신 동포 여러분께 안부 인사를 전합니다.,AIHUB_일반상식문장생성데이터_406494,None


In [8]:
filtered_df= filtered_df.dropna(subset=['text'])

In [9]:
filtered_df=filtered_df.reset_index(drop=True)

In [10]:
def cal_element(text):
    return len(text.split('. '))

In [11]:
filtered_df['len_elements'] = filtered_df['text'].apply(lambda x: len(x.split('. ')) if isinstance(x, str) else 1)

In [12]:
filtered_df['text'][0]

'아버지께 들은 이야기로 인해서 소는 나에게 경계 대상이다.'

In [13]:
filtered_df=filtered_df.reset_index(drop=True)

# Corpus 필터링 입니다.

In [26]:
#filtered_df_1 = filtered_df[(filtered_df['len_elements'] > 2)&(filtered_df['len_elements'] < 15)] # 15기준
#filtered_df_2 = filtered_df[filtered_df['len_elements'] <= 2]

############# 일반상식
filtered_df_1 = filtered_df[(filtered_df['len_elements'] >= 2)&(filtered_df['len_elements'] < 15)] # 15기준
filtered_df_2 = filtered_df[filtered_df['len_elements'] < 2]

In [27]:
filtered_df_1=filtered_df_1.reset_index(drop=True)

In [28]:
filtered_df['len_elements'].value_counts()

1    406493
2         3
Name: len_elements, dtype: int64

In [29]:
filtered_df_1

,text,doc_id,domain,len_elements
0,하나에 열중하니 아무래도. 다른 것에는 소원하게 되었다.,AIHUB_일반상식문장생성데이터_101832,None,2
1,"검은색, 붉은색. 녹색의 다채로운 쌀과, 찹쌀 등을 생산하고 있다.",AIHUB_일반상식문장생성데이터_191123,None,2
2,이 화장품 회사는 예전부터 꾸준히 1위. 자리를 지키고 있다.,AIHUB_일반상식문장생성데이터_299282,None,2


In [30]:
filtered_df_1['text'] = ['하나에 열중하니 아무래도, 다른 것에는 소원하게 되었다.','검은색, 붉은색, 녹색의 다채로운 쌀과, 찹쌀 등을 생산하고 있다.','이 화장품 회사는 예전부터 꾸준히 1위 자리를 지키고 있다.']

In [31]:
filtered_df_1['len_elements'] = 1

In [32]:
filtered_df_1

,text,doc_id,domain,len_elements
0,"하나에 열중하니 아무래도, 다른 것에는 소원하게 되었다.",AIHUB_일반상식문장생성데이터_101832,None,1
1,"검은색, 붉은색, 녹색의 다채로운 쌀과, 찹쌀 등을 생산하고 있다.",AIHUB_일반상식문장생성데이터_191123,None,1
2,이 화장품 회사는 예전부터 꾸준히 1위 자리를 지키고 있다.,AIHUB_일반상식문장생성데이터_299282,None,1


In [33]:
filtered_df_2=filtered_df_2.reset_index(drop=True)

In [34]:
filtered_df_2

,text,doc_id,domain,len_elements
0,아버지께 들은 이야기로 인해서 소는 나에게 경계 대상이다.,AIHUB_일반상식문장생성데이터_0,None,1
1,마을은 축제를 구경하기 위해 찾아온 사람들로 붐빈다.,AIHUB_일반상식문장생성데이터_1,None,1
2,그들은 우리의 시각을 자극하는 요소들을 정비했다.,AIHUB_일반상식문장생성데이터_2,None,1
3,나는 옛날 대학 시절부터 오늘날까지 이 책을 읽어 왔다.,AIHUB_일반상식문장생성데이터_3,None,1
4,농부는 농장에서 키우는 말에 고삐를 달아 끌고 갔다.,AIHUB_일반상식문장생성데이터_4,None,1
...,...,...,...,...
406488,총을 든 그들은 그에게 총격을 가해 왔다.,AIHUB_일반상식문장생성데이터_406491,None,1
406489,그가 직장을 그만두고 나서 힘들게 생활하고 있다.,AIHUB_일반상식문장생성데이터_406492,None,1
406490,그는 그 일을 아무 생각 없이 그대로 따라했다.,AIHUB_일반상식문장생성데이터_406493,None,1
406491,이번 추석에도 멀리 해외에 계신 동포 여러분께 안부 인사를 전합니다.,AIHUB_일반상식문장생성데이터_406494,None,1


In [46]:
filtered_df_2 = pd.concat([filtered_df_2, filtered_df_1], axis=0)

In [47]:
filtered_df_2=filtered_df_2.reset_index(drop=True)

# 4. Sentence order inference

- make_text_allign_data 함수는 Sentence order inference를 진행하는 함수입니다.


In [48]:
def make_text_allign_data(text):

    if not (text.endswith('.') or text.endswith('. ')):
        #print(text)

        text+='.'



    if text.endswith('.'):
        text+=' '



    my_list = text.split('. ')
    my_list_with_dots = [element + '.' for element in my_list]

    if my_list_with_dots[-1]=='.':

        my_list_with_dots=my_list_with_dots[:-1]



    random.shuffle(my_list_with_dots)


    if len(my_list_with_dots)==1:
        raise Exception("allign 리스트의 길이가 1입니다.",text)


    tmp_input='당신은 인공지능 비서입니다. 주어진 문장 리스트를 모두 활용하여 가장 정확한 단락을 생성하세요.'
    tmp_inst=f'이 문장들은 임의의 순서로 섞여 있습니다. 모든 문장을 활용하여 원본 단락의 순서와 내용을 올바른 순서로 재구성하세요.\n#문장 리스트: {my_list_with_dots}'
    
    
    if text.endswith(' '):
        tmp_out=text[:-1]

    else:
        tmp_out=text

        

    return tmp_input, tmp_inst, tmp_out



    






# 5. Last sentence prediction

-  `make_completion_data` 함수는 Last sentence prediction를 진행하는 함수입니다.


In [49]:
def make_completion_data(text):


    if not (text.endswith('.') or text.endswith('. ')):
        #print(text)

        text+='.'



    if text.endswith('. '):
        text=text

    else:
        text=text+' '



    my_list_with_dots = text.split('. ')


#    print(my_list_with_dots)


    # ㅁㅁㅁㅁㅁ. 이렇게 공백있이 끝난경우
    if my_list_with_dots[-1]=='':

        my_list_with_dots=my_list_with_dots[:-1]

        if len(my_list_with_dots)==1:
            raise Exception("Completion 텍스트를 나눈  리스트의 길이가 1입니다.",text)

        last_sentence=my_list_with_dots.pop()
        last_sentence+='.'


        
    remaining_paragraph=''




    for i in range(len(my_list_with_dots)):
        
        if i!=len(my_list_with_dots)-1:
            remaining_paragraph+=my_list_with_dots[i]+'. '

        else:
            remaining_paragraph+=my_list_with_dots[i]+'.'





    tmp_input='당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변을 생성하세요.'

    tmp_instruct=f'다음 텍스트에서 제공된 문맥을 정확히 이해하고, 마지막 문장을 자연스럽고 문맥에 맞게 완성하세요. 문장은 이전 내용과 논리적으로 연결되어야 합니다.\n#텍스트: {remaining_paragraph}'

    tmp_output=last_sentence


    return tmp_input, tmp_instruct, tmp_output
    


    

    

    
    

# 6. Mask Prediction

-  `make_text_mask_data` 함수는 Mask Prediction를 진행하는 함수입니다.


In [50]:
def make_text_mask_data(text):


    if not (text.endswith('.') or text.endswith('. ')):
        #print(text)

        text+='.'

    if text.endswith('. '):
        text=text[:-1]

    else:
        text=text


    words = re.findall(r'[가-힣]{2,}', text)
    
    random_word = random.choice(words)

    

  

    masked_text = text.replace(random_word, "<MASK>")


    tmp_input='당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.'
    tmp_instruct=f'이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를 예측해야 합니다. <MASK>가 위치한 문장의 전체 문맥을 분석하여, 문장의 나머지 내용과 일관되게 <MASK>에 들어갈 가장 적합한 단어를 답하세요.\n#텍스트: {masked_text}'
    tmp_output=random_word



    return tmp_input, tmp_instruct, tmp_output






# 4. Sentence order inference

- make_word_align 함수는 Sentence order inference를 진행하는 함수입니다.

- 다만 문장 단위가 아닌 단어 단위로 생성하는 함수입니다.


In [51]:
def make_word_align(text):

    word_lst=[]
    for word in text.split(' '):
        out = re.sub(r"[^\w\s]", "", word)
        word_lst.append(out)


    # 중복 제거하기 위해 set으로 만듬
    word_lst=set(word_lst)


    # 다시 리스트 형식으로 돌림
    word_lst=list(word_lst)


    # 랜덤하게 재배열
    random.shuffle(word_lst)


    tmp_input= '당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.'
    tmp_instruct=f'이 문제에는 문장에서 공백을 기준으로 나누고, 구두점을 제거한 무작위로 섞인 단어들이 담긴 리스트가 제공됩니다. 이 리스트의 단어를 모두 활용하여 가장 문맥상 적절한 문장을 생성하세요.\n#단어리스트: {word_lst}'
    tmp_output=text

    return tmp_input, tmp_instruct, tmp_output










# 데이터 생성 파이프라인




In [53]:
input_lst=[]
output_lst=[]
inst_lst=[]
id_lst=[]


for i in  tqdm(range(len(filtered_df_2))):
    try:

        text=filtered_df_2['text'][i]

        

        thred=random.random()



        if thred < 0.56 :

            tmp_input, tmp_instruct, tmp_output = make_word_align(text)
            tmp_id='word_allign_aihub'

        else:
            
            tmp_input, tmp_instruct, tmp_output = make_text_mask_data(text)
            tmp_id='pre_mask_aihub'




        input_lst.append(tmp_input)
        inst_lst.append(tmp_instruct)
        output_lst.append(tmp_output)
        id_lst.append(tmp_id)





    except Exception as e:
        print(f"{i}번째 행")
        print(e)
        print(filtered_df_2['text'][i])
        print("----------------------------------------")
  
    
            

In [54]:
len(filtered_df_2)

406496

In [55]:
len(output_lst)


406496

In [56]:
hub_df_2=pd.DataFrame({'input':input_lst,'instruction':inst_lst,'output':output_lst})

In [57]:
hub_df = hub_df_2.copy()

In [58]:
hub_df=hub_df.reset_index(drop=True)

In [59]:
hub_df

,input,instruction,output
0,당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.,"이 문제에는 문장에서 공백을 기준으로 나누고, 구두점을 제거한 무작위로 섞인 단어들...",아버지께 들은 이야기로 인해서 소는 나에게 경계 대상이다.
1,당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.,이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를...,위해
2,당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.,"이 문제에는 문장에서 공백을 기준으로 나누고, 구두점을 제거한 무작위로 섞인 단어들...",그들은 우리의 시각을 자극하는 요소들을 정비했다.
3,당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.,"이 문제에는 문장에서 공백을 기준으로 나누고, 구두점을 제거한 무작위로 섞인 단어들...",나는 옛날 대학 시절부터 오늘날까지 이 책을 읽어 왔다.
4,당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.,"이 문제에는 문장에서 공백을 기준으로 나누고, 구두점을 제거한 무작위로 섞인 단어들...",농부는 농장에서 키우는 말에 고삐를 달아 끌고 갔다.
...,...,...,...
406491,당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.,이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를...,계신
406492,당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.,"이 문제에는 문장에서 공백을 기준으로 나누고, 구두점을 제거한 무작위로 섞인 단어들...",옷이 너무 비싸서 친구의 선물로 살지 말지 고민이다.
406493,당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.,"이 문제에는 문장에서 공백을 기준으로 나누고, 구두점을 제거한 무작위로 섞인 단어들...","하나에 열중하니 아무래도, 다른 것에는 소원하게 되었다."
406494,당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.,이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를...,등을


In [60]:
hub_df.iloc[0,2]

'아버지께 들은 이야기로 인해서 소는 나에게 경계 대상이다.'

In [227]:
hub_df.to_parquet('./Save/일반상식문장생성데이터.parquet', engine = 'pyarrow', index = False)